In [11]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [12]:
price_data = pd.read_csv('stockPriceData-2.csv')
# Convert 'Date' to datetime type
price_data['Date'] = pd.to_datetime(price_data['Date'])

top_500 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
# Filter price data for dates after '2006-01-01' and drop duplicates
price_data.drop_duplicates(['Date', 'Symbol'], inplace=True)

company_master = pd.read_csv('Companymaster_6_12_2025.csv')

yearly = pd.read_csv('Finance_cons_pl_6_12_2025.csv')
Quaterly = pd.read_csv('Quarterly_cons_17_6_2025.csv')
Finance_bs =  pd.read_csv('Finance_cons_bs_6_12_2025.csv')
cashflow =  pd.read_csv('Finance_cons_cf_6_12_2025.csv')
ratios = pd.read_csv('Finance_cons_fr_6_12_2025.csv')

rebal_dates = pd.read_excel('GrowthDateUpdated.xlsx')

rebal_dates["Date"] = pd.to_datetime(rebal_dates["Date"])
rebal=rebal_dates['Date']

Rebalance_Qtr=list(rebal_dates['Quarter'])
Rebalance_Date=list(rebal_dates['Date'])
Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))
top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)
index_constituents_data= top_500

X = Quaterly[Quaterly['Result_Type'] == 'Q']
print(X.columns)

Index(['Fincode', 'Result_Type', 'NoOfMonths', 'Date_End', 'NET_SALES',
       'INT_ADV', 'INC_INV', 'INT_BAL', 'INT_OTHERS', 'OTHER_INCOME',
       ...
       'exc_foreign_exch_gain', 'exc_other', 'Curr_tax', 'Def_tax',
       'Fringe_benefits', 'Prior_Period_Tax', 'Mat_Credit', 'Other_Tax',
       'flag', 'notes'],
      dtype='object', length=132)


In [ ]:
ratios = ratios[['FINCODE','Year_end','ROE','ROCE','Asset_Turnover']]
ratios

,FINCODE,Year_end,ROE,ROCE,Asset_Turnover
0,100002,201112,7.287598,12.394980,1.183493
1,100002,201212,5.488083,9.822623,1.190453
2,100003,200703,18.483419,16.452193,0.979411
3,100003,200803,28.310913,27.172512,1.321627
4,100003,200903,17.085428,17.625198,1.117455
...,...,...,...,...,...
40780,320810,202303,18.633700,17.903500,2.086300
40781,320810,202403,14.292700,14.109300,1.469700
40782,320850,202203,71.466100,96.284900,2.697100
40783,320850,202303,69.304500,94.566500,3.660000


In [17]:
data = pd.merge(ratios, company_master[['FINCODE', 'SYMBOL']], left_on='FINCODE', right_on='FINCODE')

In [19]:
GrowthDate = pd.read_excel('GrowthDateUpdated.xlsx')[['Date', 'Qtr']]
GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')
# GrowthDate Mapping  
final_df =  pd.merge(data, GrowthDate, left_on='Year_end', right_on='Qtr').drop(columns=['Year_end', 'Qtr'])
final_df = final_df.sort_values(by='Date').reset_index(drop=True)
final_df

,FINCODE,ROE,ROCE,Asset_Turnover,SYMBOL,Date
0,101391,19.399672,15.858754,0.551936,NaN,2006-05-31
1,103685,99.870976,204.431949,3.948980,NaN,2006-05-31
2,131746,40.250938,23.152976,0.184410,PRAENG,2006-05-31
3,131728,32.640403,26.352459,3.422857,NaN,2006-05-31
4,103699,32.585120,32.380960,0.664836,GEODESIC,2006-05-31
...,...,...,...,...,...,...
37554,318676,41.153500,46.277900,1.743500,MONOLITH,2025-06-02
37555,100184,16.616900,22.170500,0.927300,HSCL,2025-06-02
37556,100570,23.176400,21.467600,1.206200,TATAMOTORS,2025-06-02
37557,132529,-137.722900,-64.619500,0.751300,NDTV,2025-06-02


In [20]:
price_data['Date'] = pd.to_datetime(price_data['Date'])
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df.rename(columns={'SYMBOL' : 'Symbol'},inplace=True)
merged_df = pd.merge(final_df[['Date', 'Symbol','ROE','ROCE','Asset_Turnover']], price_data, on=['Date', 'Symbol'], how='outer')
merged_df =  merged_df.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
merged_df

,Date,Symbol,ROE,ROCE,Asset_Turnover,Open,High,Low,Close,Volume,Mcap
11,1995-06-16,RELIANCE,NaN,NaN,NaN,13.590561,13.812036,13.590561,13.711366,1272550.0,125404.734065
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,13.474803,13.474803,13.194078,13.317597,580400.0,80205.346611
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,35.061728,35.061728,34.671605,34.765432,500.0,67415.040000
5,1995-06-16,GRASIM,NaN,NaN,NaN,129.684348,129.684348,129.505061,129.505061,250.0,47011.311897
3,1995-06-16,BPCL,NaN,NaN,NaN,12.497917,12.497917,12.497917,12.497917,100.0,44992.500000
...,...,...,...,...,...,...,...,...,...,...,...
7592637,2025-06-13,NSLNISP,NaN,NaN,NaN,39.150000,39.420000,38.610000,39.050000,3150428.0,114440.158443
7591922,2025-06-13,GENUSPOWER,NaN,NaN,NaN,369.950000,382.800000,368.000000,370.500000,2990813.0,112605.359198
7593462,2025-06-13,WESTLIFE,NaN,NaN,NaN,695.000000,718.950000,691.450000,716.050000,115027.0,111658.090948
7591602,2025-06-13,CCL,NaN,NaN,NaN,800.000000,841.050000,799.950000,833.700000,201301.0,111330.230424


In [22]:
merged_df[['ROE','ROCE','Asset_Turnover']] = merged_df.groupby('Symbol', group_keys=False)[['ROE','ROCE','Asset_Turnover']].apply(lambda x: x.fillna(method='ffill'))
merged_df

,Date,Symbol,ROE,ROCE,Asset_Turnover,Open,High,Low,Close,Volume,Mcap
11,1995-06-16,RELIANCE,NaN,NaN,NaN,13.590561,13.812036,13.590561,13.711366,1272550.0,125404.734065
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,13.474803,13.474803,13.194078,13.317597,580400.0,80205.346611
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,35.061728,35.061728,34.671605,34.765432,500.0,67415.040000
5,1995-06-16,GRASIM,NaN,NaN,NaN,129.684348,129.684348,129.505061,129.505061,250.0,47011.311897
3,1995-06-16,BPCL,NaN,NaN,NaN,12.497917,12.497917,12.497917,12.497917,100.0,44992.500000
...,...,...,...,...,...,...,...,...,...,...,...
7592637,2025-06-13,NSLNISP,NaN,NaN,NaN,39.150000,39.420000,38.610000,39.050000,3150428.0,114440.158443
7591922,2025-06-13,GENUSPOWER,NaN,NaN,NaN,369.950000,382.800000,368.000000,370.500000,2990813.0,112605.359198
7593462,2025-06-13,WESTLIFE,12.0699,25.6922,1.1089,695.000000,718.950000,691.450000,716.050000,115027.0,111658.090948
7591602,2025-06-13,CCL,20.7994,15.7319,0.8875,800.000000,841.050000,799.950000,833.700000,201301.0,111330.230424


In [23]:
merged_df = merged_df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Mcap'])
merged_df

,Date,Symbol,ROE,ROCE,Asset_Turnover
11,1995-06-16,RELIANCE,NaN,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN,NaN
...,...,...,...,...,...
7592637,2025-06-13,NSLNISP,NaN,NaN,NaN
7591922,2025-06-13,GENUSPOWER,NaN,NaN,NaN
7593462,2025-06-13,WESTLIFE,12.0699,25.6922,1.1089
7591602,2025-06-13,CCL,20.7994,15.7319,0.8875


In [24]:
rank_cols = [col for col in merged_df.columns if col not in ['Date', 'Symbol']]

# Apply cross-sectional percentile rank for each date
merged_df[rank_cols] = merged_df.groupby('Date')[rank_cols].transform(lambda x: x.rank(pct=True))

merged_df

,Date,Symbol,ROE,ROCE,Asset_Turnover
11,1995-06-16,RELIANCE,NaN,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN,NaN
...,...,...,...,...,...
7592637,2025-06-13,NSLNISP,NaN,NaN,NaN
7591922,2025-06-13,GENUSPOWER,NaN,NaN,NaN
7593462,2025-06-13,WESTLIFE,0.270784,0.719715,0.696742
7591602,2025-06-13,CCL,0.672209,0.418052,0.537594


In [25]:
merged_df.to_csv('New Quality.csv', index=False)